#### Dowload :

In [ ]:
!pip install -q langchain
!pip install -q langchain-community

!pip install -q sentence-transformers

!pip install -q pinecone-client

!pip install -q langchain-pinecone

!pip install -q python-dotenv

!pip install -q pypdf

!pip install -q transformers einops accelerate langchain bitsandbytes

!nvidia-smi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 8.1 MB/s eta 0:00:00
Wed Aug  7 10:53:30 2024       
+-------

In [ ]:
from langchain_community.document_loaders import PyPDFLoader , DirectoryLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore


from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.llms import CTransformers

In [ ]:
from transformers import AutoTokenizer, EsmModel
import torch

In [ ]:
PINECONE_API_KEY = "........."
# PINECONE_API_ENV = ""
index_name = "........."

#### Reading Data & Create text chunks ::

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
%cd 'drive/MyDrive/Colab_Notebooks/model-2'

/content/drive/MyDrive/Colab_Notebooks/model-2


In [ ]:
import pandas as pd
df = pd.read_csv('aminos.csv')
df

,protein Names,Cell target,RNA sequence,Paper research Link
0,Nanog,neurone,MSVDPACPQS LPCFEASDCK ESSPMPVICG PEENYPSLQM S...,"Functional Expression Cloning of Nanog, a Plur..."
1,Oct4 (POU5F1),neurone,MSVDPACPQS LPCFEASDCK ESSPMPVICG PEENYPSL...,NaN
2,Sox2,neurone,MYNMMETELK PPGPQQTSGG GGGNSTAAAA GGNQKNSPDR V...,Multipotent cell lineages in early mouse devel...
3,Ngn2 (Neurogenin 2),neurone,MFVKSETLEL KEEEDVLVLL GSASPALAAL TPLSSSADEE E...,"Identification of neurogenin, a Vertebrate Neu..."
4,Pax6,neurone/yeux,MQNSHSGVNQ LGGVFVNGRP LPDSTRQKIV ELAHSGARPC DI...,Pax6; a pleiotropic player in development
5,Nestin,neurone,MEGCMGEESF QMWELNRRLE AYLARVKALE EQNELLSAEL G...,Isolation of Neural Stem Cells from Whole Brai...
6,Brachyury,muscle / os tissu,MSSPGTESAG KSLQYRVDHL LSAVENELQA GSEKGDPTER E...,T-box Genes in Early Embryogenesis - PMC
7,GATA4,coeur,MYQSLAMAAN HGPPPGAYEA GGPGAFMHGA GAASSPVYVP T...,Myocardial molecular biology: an introduction ...
8,Sox17,poumon/ foie/ pancréas,MSSPDAGYAS DDQSQTQSAL PAVMAGLGPC PWAESLSPIG D...,Depletion of definitive gut endoderm in Sox17-...
9,Runx1,Sang,MRIPVDASTS RRFTPPSTAL SPGKMSEALP LGAPDAGAAL A...,Expression pattern of Runt-related transcripti...


In [ ]:
amino_acid_list = df['RNA sequence'].tolist()

In [ ]:
len(amino_acid_list)

11

In [ ]:
# Remove spaces and newlines
processed_list = [''.join(sequence.split()) for sequence in amino_acid_list]

# Remove duplicates by converting the list to a set and back to a list
text_chunks = list(set(processed_list))

print(len(text_chunks))

10


In [ ]:
text_chunks

['MSSPDAGYASDDQSQTQSALPAVMAGLGPCPWAESLSPIGDMKVKGEAPANSGAPAGAAGRAKGESRIRRPMNAFMVWAKDERKRLAQQNPDLHNAELSKMLGKSWKALTLAEKRPFVEEAERLRVQHMQDHPNYKYRPRRRKQVKRLKRVEGGFLHGLAEPQAAALGPEGGRVAMDGLGLQFPEQGFPAGPPLLPPHMGGHYRDCQSLGAPPLDGYPLPTPDTSPLDGVDPDPAFFAAPMPGDCPAAGTYSYAQVSDYAGPPEPPAGPMHPRLGPEPAGPSIPGLLAPPSALHVYYGAMGSPGAGGGRGFQMQPQHQHQHQHQHHPPGPGQPSPPPEALPCRDGTDPSQPAELLGEVDRTEFEQYLHFVCKPEMGLPYQGHDSGVNLPDSHGAISSVVSDASSAVYYCNYPDV',
 'MQNSHSGVNQLGGVFVNGRPLPDSTRQKIVELAHSGARPCDISRILQVSNGCVSKILGRYYETGSIRPRAIGGSKPRVATPEVVSKIAQYKRECPSIFAWEIRDRLLSEGVCTNDNIPSVSSINRVLRNLASEKQQMGADGMYDKLRMLNGQTGSWGTRPGWYPGTSVPGQPTQDGCQQQEGGGENTNSISSNGEDSDEAQMRLQLKRKLQRNRTSFTQEQIEALEKEFERTHYPDVFARERLAAKIDLPEARIQVWFSNRRAKWRREEKLRNQRRQASNTPSHIPISSSFSTSVYQPIPQPTTPVSSFTSGSMLGRTDTALTNTYSALPPMPSFTMANNLPMQPPVPSQTSSYSCMLPTSPSVNGRSYDTYTPPHMQTHMNSQPMGTSGTTSTGLISPGVSVPVQVPGSEPDMSQYWPRLQ',
 'MEGCMGEESFQMWELNRRLEAYLARVKALEEQNELLSAELGGLRAQSADTSWRAHADDELAALRALVDQRWREKHAAEVARDNLAEELEGVAGRCQQLRLARERTTEEVARNRRAVEAEKCARAWLSSQVAELERELEALRVAHEEERVGLN

#### Download Embeddings :

In [ ]:
# class ESM2Embeddings:
#     def __init__(self, model_name="facebook/esm2_t33_650M_UR50D"):
#         self.tokenizer = AutoTokenizer.from_pretrained(model_name)
#         self.model = EsmModel.from_pretrained(model_name)

#     def embed_sequence(self, sequence):
#         inputs = self.tokenizer(sequence, return_tensors="pt")
#         with torch.no_grad():
#             outputs = self.model(**inputs)
#             embeddings = outputs.last_hidden_state
#         # Average the embeddings across the sequence length (excluding special tokens)
#         sequence_embedding = embeddings[0, 1:-1, :].mean(dim=0)  # Shape: (d,)
#         return sequence_embedding.numpy()

In [ ]:
# embeddings = ESM2Embeddings()

In [ ]:
from typing import List

class ESM2Embeddings:
    def __init__(self, model_name: str):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = EsmModel.from_pretrained(model_name)

    def embed_documents(self, documents: List[str]) -> List[List[float]]:
        embeddings = []
        for doc in documents:
            inputs = self.tokenizer(doc, return_tensors="pt", truncation=True, padding=True)
            with torch.no_grad():
                outputs = self.model(**inputs)
                doc_embedding = outputs.last_hidden_state.mean(dim=1).squeeze().tolist()
            embeddings.append(doc_embedding)
        return embeddings

    def embed_query(self, query: str) -> List[float]:
        inputs = self.tokenizer(query, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            outputs = self.model(**inputs)
            query_embedding = outputs.last_hidden_state.mean(dim=1).squeeze().tolist()
        return query_embedding

In [ ]:
embedding = ESM2Embeddings(model_name="facebook/esm2_t33_650M_UR50D")

##### test test :

In [ ]:
# # Load the tokenizer and model
# model_name = "facebook/esm2_t33_650M_UR50D"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = EsmModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.61G [00:00<?, ?B/s]

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t33_650M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# sequence = "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG"

In [ ]:
# inputs = tokenizer(sequence, return_tensors="pt")

In [ ]:
# with torch.no_grad():
#     outputs = model(**inputs)
#     embeddings = outputs.last_hidden_state

In [ ]:
# # Average the embeddings across the sequence length (excluding special tokens)
# # The first and last tokens are usually special tokens (e.g., <CLS> and <SEP>)
# # So we average from index 1 to -1
# sequence_embedding = embeddings[0, 1:-1, :].mean(dim=0)  # Shape: (d,)

In [ ]:
# # Print the resulting embedding
# print(sequence_embedding)
# print("Embedding shape:", sequence_embedding.shape)  # Should be (d,)

tensor([ 0.0614, -0.0687,  0.0430,  ..., -0.1642, -0.0678,  0.0446])
Embedding shape: torch.Size([1280])


#### Embedding chunks and storing them into pinecone :

In [ ]:
pc = Pinecone(api_key=PINECONE_API_KEY) # ,environment=PINECONE_API_ENV

In [ ]:
if index_name not in pc.list_indexes().names():
    print(index_name , " Does not exist !")
    pc.create_index(
        name=index_name,
        dimension=1280,
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

In [ ]:
index = pc.Index(index_name)
index.describe_index_stats()
# index.delete(delete_all=True, namespace='......')

{'dimension': 1280,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 10}},
 'total_vector_count': 10}

In [ ]:
vectorstore = PineconeVectorStore(
    index=index,
    pinecone_api_key = PINECONE_API_KEY,
    embedding=embedding,
    index_name=index_name
)

In [ ]:
# vectorstore.add_texts(text_chunks)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


['5cf13156-46b2-44bb-b774-e1a21ed67a7d',
 '176abc72-1979-4b5a-842e-bd08fd6a630a',
 '93f442f5-7b6a-40e1-ba82-199b174dea6a',
 '2a71849c-5a2a-402b-ab83-3a9f14397a24',
 'c22c0ec2-59ef-446d-8867-44753a210479',
 'be62bcd4-fc25-4b4f-8ea0-22241ea22532',
 '8ea3e997-4f90-4100-9df8-154f1a402bce',
 'ec32d047-4815-421e-8d62-44168fcc5608',
 'c0ddcd7d-3247-4ad3-84e2-2106d133fa6c',
 '9907ebc8-f266-4ff7-86d9-3cc9f9b761bc']

In [ ]:
index.describe_index_stats()

{'dimension': 1280,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 10}},
 'total_vector_count': 10}

#### To amino acid & Similarity Search :

In [ ]:
# DNA to RNA Transcription
def transcribe_dna_to_rna(dna_seq):
    return dna_seq.replace('T', 'U')

# RNA to Amino Acid Translation
def translate_rna_to_protein(rna_seq):
    codon_table = {
        'UUU': 'F', 'UUC': 'F', 'UUA': 'L', 'UUG': 'L',
        'UCU': 'S', 'UCC': 'S', 'UCA': 'S', 'UCG': 'S',
        'UAU': 'Y', 'UAC': 'Y', 'UAA': '*', 'UAG': '*',
        'UGU': 'C', 'UGC': 'C', 'UGA': '*', 'UGG': 'W',
        'CUU': 'L', 'CUC': 'L', 'CUA': 'L', 'CUG': 'L',
        'CCU': 'P', 'CCC': 'P', 'CCA': 'P', 'CCG': 'P',
        'CAU': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q',
        'CGU': 'R', 'CGC': 'R', 'CGA': 'R', 'CGG': 'R',
        'AUU': 'I', 'AUC': 'I', 'AUA': 'I', 'AUG': 'M',
        'ACU': 'T', 'ACC': 'T', 'ACA': 'T', 'ACG': 'T',
        'AAU': 'N', 'AAC': 'N', 'AAA': 'K', 'AAG': 'K',
        'AGU': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R',
        'GUU': 'V', 'GUC': 'V', 'GUA': 'V', 'GUG': 'V',
        'GCU': 'A', 'GCC': 'A', 'GCA': 'A', 'GCG': 'A',
        'GAU': 'D', 'GAC': 'D', 'GAA': 'E', 'GAG': 'E',
        'GGU': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G'
    }
    protein_seq = ""
    for i in range(0, len(rna_seq) - 2, 3):
        codon = rna_seq[i:i+3]
        protein_seq += codon_table.get(codon, 'X')  # 'X' for unknown codons
    return protein_seq

# Main Function to Convert DNA to Protein
def dna_to_protein(dna_seq):
    rna_seq = transcribe_dna_to_rna(dna_seq)
    protein_seq = translate_rna_to_protein(rna_seq)
    return protein_seq

In [ ]:
dna_seq =  "CTTTATTGATTGGTGTGCCGTTTTCTCTGGAAGCCTCTTAAGAACACAGTGGCACAGGCTGGGTGGAGCCGTCCC"
protein_seq = dna_to_protein(dna_seq)

In [ ]:
len(protein_seq)

In [ ]:
pc = Pinecone(api_key=PINECONE_API_KEY) # ,environment=PINECONE_API_ENV
index = pc.Index(index_name)

In [ ]:
queryvectorstore = PineconeVectorStore(
    index=index,
    pinecone_api_key = PINECONE_API_KEY,
    embedding=embedding,
    index_name=index_name
)

In [ ]:
query = protein_seq
docs = queryvectorstore.similarity_search(query, k=3)
print("Result", docs)

Result [Document(page_content='MRIPVDASTSRRFTPPSTALSPGKMSEALPLGAPDAGAALAGKLRSGDRSMVEVLADHPGELVRTDSPNFLCSVLPTHWRCNKTLPIAFKVVALGDVPDGTLVTVMAGNDENYSAELRNATAAMKNQVARFNDLRFVGRSGRGKSFTLTITVFTNPPQVATYHRAIKITVDGPREPRRHRQKLDDQTKPGSLSFSERLSELEQLRRTAMRVSPHHPAPTPNPRASLNHSTAFNPQPQSQMQDTRQIQPSPPWSYDQSYQYLGSIASPSVHPATPISPGRASGMTTLSAELSSRLSTAPDLTAFSDPRQFPALPSISDPRMHYPGAFTYSPTPVTSGIGIGMSAMGSATRYHTYLPPPYPGSSQAQGGPFQASSPSYHLYYGASAGSYQFSMVGGERSPPRILPPCTNASTGSALLNPSLPNQSDVVEAEGSHSNSPTNMAPSARLEEAVWRPY'), Document(page_content='MSSPDAGYASDDQSQTQSALPAVMAGLGPCPWAESLSPIGDMKVKGEAPANSGAPAGAAGRAKGESRIRRPMNAFMVWAKDERKRLAQQNPDLHNAELSKMLGKSWKALTLAEKRPFVEEAERLRVQHMQDHPNYKYRPRRRKQVKRLKRVEGGFLHGLAEPQAAALGPEGGRVAMDGLGLQFPEQGFPAGPPLLPPHMGGHYRDCQSLGAPPLDGYPLPTPDTSPLDGVDPDPAFFAAPMPGDCPAAGTYSYAQVSDYAGPPEPPAGPMHPRLGPEPAGPSIPGLLAPPSALHVYYGAMGSPGAGGGRGFQMQPQHQHQHQHQHHPPGPGQPSPPPEALPCRDGTDPSQPAELLGEVDRTEFEQYLHFVCKPEMGLPYQGHDSGVNLPDSHGAISSVVSDASSAVYYCNYPDV'), Document(page_content='MSSPGTESAGKSLQYRVDHLLSAVENELQAGSEKGDPTERELRVGLEE

#### Using the llama model:

In [ ]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                model_type="llama",
                config={'max_new_tokens':512,
                        'temperature':0.8})

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=queryvectorstore.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa.invoke({"query": user_input})
    print("Response : ", result["result"])